In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [2]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [6]:
driver = webdriver.Chrome(driver_name)
driver.get(url)
time.sleep(2)

TOTAL_COUNT = 1000
START_COUNT = 0
SUB_COUNT = 0
for i in range(START_COUNT):
    driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
    time.sleep(3)
    
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
current = 0
for k in range(SUB_COUNT):
    current += 1
    lis[current].click()
    time.sleep(3)
    lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

In [23]:
TOTAL_COUNT = 10

In [13]:
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
time.sleep(3)

In [27]:

driver = webdriver.Chrome(driver_name)

driver.get(url)
time.sleep(2)
ORDER = 'movie03'
n_page = 0
# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
# print(lis)

end, current = len(lis), 0
lines = []

while True:

    try:
        trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')
    except:
        print('nPgae = ', n_page)
        break

    for tr in trs:

        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(2)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > p').text
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')

        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')

        # 코드 정보
        code = dds[0].text
        summering = dds[3].text
        # 개봉일
        first_day = dds[-6].text
        # 제작년도
        production_year = dds[-5].text
            
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div')
        movie_director, star_actor = '', ''
        find_count = 0

        # 감독 : 있는 경우 : 첫 번째에 감독이 없으면 감독이 없는 데이터다
        if len(tmp) > 0 :
            nindex = tmp[0].text.find('감독')
            if nindex >= 0:
                movie_director = tmp[0].text[nindex + 2:].strip()
                find_count += 1

        # 주연 배우
        for tp in tmp[find_count:]:

            text = tp.text.split('\n')[-1]
            if text.find('[주연]') >=0:
                star_actor = tp.text[tp.text.find('[주연]') + 4 : ].strip()
                # print(star_actor)
                break

        # print(title, movie_director, star_actor)
        lines.append({'code': code, 'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})

        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
        time.sleep(2)


    current += 1
    n_page += 1

    # if n_page == TOTAL_COUNT: break

    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(3)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            print('except2 nPgae = ', n_page)
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        # print(lis)
        lis[current].click()
        time.sleep(3)

df = pd.DataFrame(lines)

df.to_csv(f'../static/data/{ORDER}.csv', index=False)

극장판 도라에몽: 진구와 하늘의 유토피아 (Doraemon the Movie: Nobita`s Sky Utopia)  윤아영(도라에몽) | 김정아(진구) | 이현주(비실이) | 조현정(이슬이) | 최낙윤(퉁퉁이)
어디로 가고 싶으신가요 (Where Would You Like to Go?) 김희정 박하선 | 김남희 | 전석호 | 문우진
브레이크 아웃 (Breakout) 브랜던 슬래글 톰 시즈모어
욕망의 그림자 (Shades of Desire)  
하얀봄  
퀴어 마이 프렌즈 (Queer My Friends) 서아현 서아현
빈틈없는 사이 (MY WORST NEIGHBOR) 이우철 이지훈 | 한승연
콘크리트 유토피아 엄태화 
비밀의 언덕 (The Hill of Secrets) 이지은 
몸값part.2 (Bargain part.2) 전우성 진선규(형수) | 전종서(주영)
몸값part.1 (Bargain part.1) 전우성 진선규(형수) | 전종서(주영)
그녀의 취미생활 (Her Hobby) 하명미 정이서 | 김혜나
수능, 출제의 비밀 이용재 
극장판 아름다운 그: 이터널 (My Beautiful Man ~Eternal~) 사카이 마이 하기와라 리쿠 | 야기 유세이
핑크퐁 시네마 콘서트3 : 진저브레드맨을 잡아라 (Pinkfong Sing-Along Movie3: Catch the Gingerbread Man) 변희선 조경이(핑크퐁) | 이소은(호기) | 박요한(진저브레드맨) | 김새해(제니) | 장예나(아기상어)
비공식작전 (Ransomed) 김성훈 하정우 | 주지훈
젖은 백합 (When the rain falls) 카네코 슈스케 
슬기로운 아내 수업 (How to Be a Good Wife) 마르탱 프로보스트 줄리엣 비노쉬 | 욜랭드 모로 | 노에미 르보브스키 | 에두아르 바에르 | 프랑수아 베를레앙
익스트림 페스티벌 (Extreme Festival) 김홍기 김재화 | 조민재 | 박강섭
슈퍼 키드 헤일리 라스무스 A. 실버르센 신은수
명탐정코난: 흑철의 어영 (D

In [17]:
len(lines)

343

In [19]:
df = pd.DataFrame(lines)

In [34]:
driver.close()

In [20]:
df.tail()

,code,title,genre,summering,first_day,production_year,movie_director,star_actor,img,synopsis
338,20224632,물괴: 검은숲의 비밀 (Water Monster 2),액션(1),"장편 | 일반영화 | 액션, 판타지 | 84분 29초 | 중국",해당정보없음,2021년,샹추량,,https://www.kobis.or.kr/common/mast/movie/2022...,저주받은 괴물이 깨어난다!\n\n실종된 동생을 찾기 위해 검은 숲속 외딴 마을로 간...
339,20217427,메모리: 조작살인 (Memory),미스터리,장편 | 독립영화 | 미스터리 | 85분 51초 | 청소년관람불가 | 한국,2021-12-29,2021년,[조] 황경식,,https://www.kobis.or.kr/common/mast/movie/2021...,"남편의 실종, 그리고 벌어지는 이상한 사건들\n기억하고 있는 것이 모두 사실은 아니..."
340,20224181,밴디다스 : 놈의 심장을 뜯어라 (My Body Will Bury You),액션,장편 | 일반영화 | 액션 | 116분 14초 | 이탈리아,해당정보없음,2021년,,,https://www.kobis.or.kr/common/mast/movie/2022...,"무법 지대가 되어버린 땅에서 네 명의 여자들로 이루어진 도적단, 드루데 여단은\n자..."
341,20211931,아이돌레시피 (Idol Recipe),코미디,장편 | 독립영화 | 코미디 | 102분 3초 | 15세이상관람가 | 한국,2022-03-17,2021년,[조] 손혜인,,https://www.kobis.or.kr/common/mast/movie/2022...,내 인생 목표가 엔터 탈출이다!\n\n모든 열정이 식어버린 ‘15년차 아이돌 매니저...
342,20210818,"너도 평범하지 않아 (You're not normal, either!)",코미디(1),"장편 | 일반영화 | 코미디, 멜로/로맨스 | 99분 16초 | 12세이상관람가 | 일본",해당정보없음,2021년,마에다 코지,,https://www.kobis.or.kr/common/mast/movie/2022...,"세상의 평범과는 거리가 먼, 수학밖에 모르는 입시학원 강사 '오노 야스오미'\n여자..."


In [21]:
df.to_csv(f'../static/data/{ORDER}.csv', index=False)